In [1]:
import pandas as pd

In [3]:
data=pd.read_csv("/content/spam_ham_dataset.csv")
data=pd.DataFrame(data.head(400))

In [4]:
from sklearn.model_selection import train_test_split
train_data, test_data=train_test_split(data, test_size=0.2, random_state=42)

In [5]:
train_data.head()

,Unnamed: 0,label,text,label_num
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
18,1629,ham,Subject: meter variances - ua 4 clean - up\r\n...,0
202,2035,ham,"Subject: enron / hpl actuals for november 13 ,...",0
250,3828,spam,Subject: ! ebay reguiar verification\r\n% word...,1
274,3728,spam,Subject: get a bu _ lky p ^ 0 le * dcrgvabyssy...,1


In [6]:
from transformers import BertTokenizer
checkpoint="bert-base-uncased"
tokenizer=BertTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
train_encoding=tokenizer(list(train_data['text']), truncation=True, padding=True,max_length=512)
test_encoding=tokenizer(list(test_data['text']),truncation=True, padding=True, max_length=512)

In [8]:
import torch
from torch.utils.data import Dataset

class EmailDataset(Dataset):
  def __init__(self,encodings,labels):
    self.encodings=encodings
    self.labels=labels

  def __getitem__(self,idx):
    item={key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels']=torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

In [9]:
train_dataset=EmailDataset(train_encoding,train_data['label_num'].tolist())
test_dataset=EmailDataset(test_encoding, test_data['label_num'].tolist())

In [10]:
from torch.utils.data import DataLoader
train_dataloader=DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader=DataLoader(test_dataset, batch_size=8,shuffle=True)


In [11]:
from transformers import BertForSequenceClassification
model=BertForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [13]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [14]:
from torch.optim import AdamW
from transformers import get_scheduler

optimizer=AdamW(model.parameters(), lr=5e-5)
num_epochs=3
num_training_steps=num_epochs*len(train_dataloader)

scheduler=get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [15]:
model.train()
for epoch in range(num_epochs):
  for batch in train_dataloader:
    batch={k: v.to(device) for k, v in batch.items()}
    outputs=model(**batch)
    loss=outputs.loss
    loss.backward()

    optimizer.step()
    scheduler.step()

    optimizer.zero_grad()

    print(f"epoch:{epoch} , Loss: {loss.item()}")

model.eval()
total_eval_acc=0
total_eval_loss=0




epoch:0 , Loss: 0.6562095880508423
epoch:0 , Loss: 0.5878246426582336
epoch:0 , Loss: 0.5585287809371948
epoch:0 , Loss: 0.5177960991859436
epoch:0 , Loss: 0.41227805614471436
epoch:0 , Loss: 0.5577144026756287
epoch:0 , Loss: 0.8152490854263306
epoch:0 , Loss: 0.522159218788147
epoch:0 , Loss: 0.46198996901512146
epoch:0 , Loss: 0.4984534680843353
epoch:0 , Loss: 0.341444730758667
epoch:0 , Loss: 0.29271289706230164
epoch:0 , Loss: 1.045074701309204
epoch:0 , Loss: 0.8179451823234558
epoch:0 , Loss: 0.4794549345970154
epoch:0 , Loss: 0.33718225359916687
epoch:0 , Loss: 0.6974642276763916
epoch:0 , Loss: 0.4838237762451172
epoch:0 , Loss: 0.36378198862075806
epoch:0 , Loss: 0.2437729388475418
epoch:0 , Loss: 0.27913033962249756
epoch:0 , Loss: 0.2678689956665039
epoch:0 , Loss: 0.34435322880744934
epoch:0 , Loss: 0.3591512441635132
epoch:0 , Loss: 0.3313770592212677
epoch:0 , Loss: 0.17251160740852356
epoch:0 , Loss: 0.15994489192962646
epoch:0 , Loss: 0.5524463653564453
epoch:0 , Loss

In [18]:
for batch in test_dataloader:
  batch={k:v.to(device) for k,v in batch.items()}

  with torch.no_grad():
    outputs=model(**batch)

  loss=outputs.loss
  total_eval_loss +=loss.item()

  logits=outputs.logits
  predictions=torch.argmax(logits,dim=-1)
  total_eval_acc +=(predictions == batch['labels']).float().mean()

avg_test_loss = total_eval_loss / len(test_dataloader)
avg_test_accuracy = total_eval_acc / len(test_dataloader)
print(f"Test Loss: {avg_test_loss}, Test Accuracy: {avg_test_accuracy}")

Test Loss: 0.1440565999597311, Test Accuracy: 0.949999988079071


In [16]:

# Move model to evaluation mode
model.eval()

# List to store all prediction entries
all_predictions_data = []

# Evaluate the model
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

    input_ids = batch['input_ids'].cpu().numpy()
    labels = batch['labels'].cpu().numpy()

    # Store all predictions with decoded texts
    for ids, pred, label in zip(input_ids, predictions.cpu().numpy(), labels):
        decoded_text = tokenizer.decode(ids, skip_special_tokens=True)
        all_predictions_data.append({
            'decoded_text': decoded_text,
            'predicted_label': pred,
            'true_label': label
        })

# Create a DataFrame from the collected data
predictions_df = pd.DataFrame(all_predictions_data)

# Print or save the DataFrame
#print(predictions_df.head())  # Print first few rows
#predictions_df.to_csv('all_predictions.csv', index=False)  # Save to CSV file


In [20]:
predictions_df.head()

,decoded_text,predicted_label,true_label
0,subject : 23 rd noms - - - - - - - - - - - - -...,0,0
1,"subject : c, | alis soft ` tabs, fast shipping...",1,1
2,"subject : meters daren, is there any industria...",0,0
3,subject : revised avails for 12 / 00 as noted ...,0,0
4,subject : low rate credit credit problems? no ...,1,1


In [17]:
pd.crosstab(predictions_df['predicted_label'],predictions_df['true_label'])

true_label,0,1
predicted_label,,
0,59,3
1,1,17


In [19]:
predictions_df.shape

(80, 3)